# Uber Pickups - Streamlit App

This notebook contains example to create a Streamlit app using Uber pickup data

The code needs to be saved as a .py file and can be run simply using 
$streamlit run <code_name.py>

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np



### Give the title to the app

In [ ]:
st.title('Uber pickups in NYC')

In [ ]:
# Some data specific globals - 

DATE_COLUMN = 'date/time'
DATA_URL = ('https://s3-us-west-2.amazonaws.com/'
         'streamlit-demo-data/uber-raw-data-sep14.csv.gz')


### Download and cache the data

We don't want to download the data everytime the app is reloaded, instead we'll use built-in caching decorator

In [ ]:
@st.cache
def load_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows)
    lowercase = lambda x: str(x).lower()
    data.rename(lowercase, axis='columns', inplace=True)
    data[DATE_COLUMN] = pd.to_datetime(data[DATE_COLUMN])
    return data

# Create a text element and let the reader know the data is loading.
data_load_state = st.text('Loading data...')
# Load 10,000 rows of data into the dataframe.
data = load_data(10000)
# Notify the reader that the data was successfully loaded.
data_load_state.text("Done! (using st.cache)")

### Preview data

In [ ]:
# As per the documentation, st.write can take multiple types of data and render on the browser.
# Here, we are passing a dataframe
st.subheader('Raw data')
st.write(data)

### Create a Bar Chart

In [ ]:
# To start, let's add a subheader just below the raw data section:
st.subheader('Number of pickups by hour')
# Use NumPy to generate a histogram that breaks down pickup times binned by hour
hist_values = np.histogram(
    data[DATE_COLUMN].dt.hour, bins=24, range=(0,24))[0]
# Use built-in bar_chart to display the bar chart
st.bar_chart(hist_values)

### Plot data on a map

st.map() renders fully interactive map. That's pretty impressive!

In [ ]:
st.subheader('Map of all pickups')
st.map(data)

In [ ]:
# Plot the map at a certain time - we found that the pickups were most busy at 5PM, let's filter the data and plot

hour_to_filter = 17
filtered_data = data[data[DATE_COLUMN].dt.hour == hour_to_filter]
st.subheader(f'Map of all pickups at {hour_to_filter}:00')
st.map(filtered_data)

In [ ]:
# Plot a dynamic map - Let user select the hour of the day and use that to filter the data and plot on the map

hour_to_filter = st.slider('hour', 0, 23, 17)
filtered_data = data[data[DATE_COLUMN].dt.hour == hour_to_filter]
st.subheader(f'Dynamic Map of all pickups at {hour_to_filter}:00')
st.map(filtered_data)

## Use a button to toggle data
Sliders are just one way to dynamically change the composition of your app. Let's use the st.checkbox function to add a checkbox to your app. We'll use this checkbox to show/hide the raw data table at the top of your app.

Note that although we are using multiple sub-headers, this is just for demo. Ideally you'd want only 1 section for data preview

In [ ]:
if st.checkbox('Show raw data'):
    st.subheader('Another preview of the raw data')
    st.write(data)